<a href="https://colab.research.google.com/github/JeongHanJun/-/blob/master/MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch.utils
import torch.optim as optim
 
import torchvision
import torchvision.transforms as transforms
 
import pandas as pd
from collections import OrderedDict
from IPython.display import clear_output
 
class NN(nn.Module):    #NN = Neural Network = 신경망
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=28*28, out_features=200)   # MNIST 그림의 크기 28x28 , 200개의 노드  = Input Layer
        self.fc2 = nn.Linear(in_features=200, out_features=200)     # First Layer -> Second Layer
        self.fc3 = nn.Linear(in_features=200, out_features=10)      # Second Layer -> Output Layer
        
    def forward(self, x):           # 학습 진행 방향
        x = F.relu(self.fc1(x))     # fc1 ->
        x = F.relu(self.fc2(x))     # fc1 -> fc2 ->
        x = self.fc3(x)             # fc1 -> fc2 -> fc3         relu max(0, x) , sigmoid는 0과 1 사이값 
        return F.log_softmax(x) 
        '''
            활성화 함수 relu 와 sigmoid
 
            relu 는 간단히 max(0,x) <0 이면 0출력 , >0 이면 그 값 출력
 
            sigmoid = 1 / (1+e^-x)  기함수형태 , (0,1/2) 대칭 , tanh 형태
            최소 제곱법에 대해 분산, 표준편차를 알수 있음
             0 <= 확률 P <= 1 에 대해  y = f(x1.x2.x3....) 변수가 많은 식에 대한 y의 값은 무의미한 값을 가지게 되는 경우가 많다.
             (종속변수 y에 대해 선형모델을 적용했을때 최소제곱직선 기준 떨어진 데이터들이 많으므로 무의미하다.)
             직선 형태를 선형 로그 (tanh 형태)로 변형해서 데이터들의 오차를 줄이려는 것
            torch.nn 의 log_softmax 는 신경망 말단 노드들의 결과값을 확률개념으로 해석하기 위해 log를 취한것 
            ( p / 1-p ) -(log , 정의영ㄱ/치역확인)> (mx+b) -(실수 전범위 이므로 의미 있는 선형 모델 , p에 대해 정리하면)> (1/1+e^-x)
 
            hidden layer 즉 학습과정에서 relu를 쓰고 마지막 output에서는 sigmoid를 쓰는게 효과적
        '''
 
net= NN()
 
learning_rate = 0.01    # 학습률 0.01로 임의 지정 
batch_size =100         # batch 는 한번에 처리하는 사진의 수 batch_size = 100 이면 한번에 100장씩 처리한다는 뜻
 
optimizer = optim.SGD(net.parameters(), lr = learning_rate, momentum= 0.9)  # 확률적 경사 하강범 ( momentom = 양수의 실수값, 가속도)  w += lr * gradient
criterion= nn.NLLLoss()         # NLLlose log_softMax 에 대한 결과값 (교차 엔트로피 손실 연산?)
 
train_set = torchvision.datasets.FashionMNIST(  # data set을 받아옴
    root = './data/FashionMNIST',
    train = True,
    download = True,
    transform = transforms.Compose([
        transforms.ToTensor()
    ])
)
 
loader = torch.utils.data.DataLoader(train_set, batch_size = batch_size)    # 위의 data를 받아옴
 
epochs = 5  # epoch는 전체 data set에 대해 몇번 학습(forward)을 돌릴것인가
 
pd_results= []  # list형태로 출력
 
for epoch in range(epochs): # 전체학습을 epochs 만큼 반복
    batch_idx = 0           # 변수 초기화
    tot_num = 0
    correct_num = 0
    for batch in loader:
        batch_idx += 1  # 1개씩 늘리면서 진행
 
        images = batch[0]
        labels = batch[1]
 
        images = images.view(-1, 28*28) # view(-1, size) 에서 왜 -1인가? -> (batch_size, 784)
        
        optimizer.zero_grad()   # gradient를 0 으로 해야 반복할때 오버래핑이 안생김
 
        net_out= net(images)
 
        loss = criterion(net_out, labels)   # 손실률
 
        loss.backward()
        optimizer.step()
 
        for i in range(len(labels)):
            tot_num += 1
            pred = torch.max(net_out[i], 0)[1]
            correct_num += pred.eq(labels[i]).sum()
        
        if batch_idx % 100 == 0:                                # data 양이 많으니 100배수 에서 출력
            results = OrderedDict()
            results['epoch'] = epoch
            results['batch_idx'] = batch_idx
            results['loss'] = loss.item()
            results['accuracy'] = 100.*correct_num.item()/tot_num
            pd_results.append(results)
            df = pd.DataFrame.from_dict(pd_results, orient = 'columns')
 
            clear_output(wait = True)
            display(df)
            

,epoch,batch_idx,loss,accuracy
0,0,100,0.948239,48.480000
1,0,200,0.905982,59.920000
2,0,300,0.582087,65.663333
3,0,400,0.559721,69.357500
4,0,500,0.632570,71.798000
5,0,600,0.496619,73.315000
6,1,100,0.404887,82.540000
7,1,200,0.510672,82.765000
8,1,300,0.493226,83.030000
9,1,400,0.485011,83.362500


In [ ]:
import numpy as np

In [ ]:
a = np.ones([28,28])

In [ ]:
a.shape

(28, 28)

In [ ]:
a.reshape(-1, 28*28).shape

(1, 784)